# Document advanced parsing and chunking

This notebook was tested on a SageMaker Studio Notebook `Data Science 3.0` kernel and  `ml.t3.xlarge` instance.


## Step 1: Setup

Install required packages

In [ ]:
%pip install requests-aws4auth
%pip install opensearch-py
%pip install -U retrying==1.3.4
%pip install -U boto3

Restart the Kernel \
Click **kernel** on the top bar and **Restart Kernel**. Continue with the cells below.

In [4]:
import boto3
from botocore.exceptions import ClientError
import pprint
from bedrockhelper import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
import json
import os

s3=boto3.client("s3")
from botocore.config import Config
config = Config(
    read_timeout=600, 
    retries = dict(
        max_attempts = 5 
    )
)

suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
account_id = sts_client.get_caller_identity()["Account"]
s3_suffix = f"{region_name}-{account_id}"
bucket_name = f'bedrock-kb-{s3_suffix}' # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
# Check if bucket exists, and if not create S3 bucket for knowledge base data source
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f'Bucket {bucket_name} Exists')
except ClientError as e:
    print(f'Creating bucket {bucket_name}')
    if region_name == "us-east-1":
        s3bucket = s3.create_bucket(
            Bucket=bucket_name)
    else:
        s3bucket = s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region_name }
    )

In [ ]:
%store bucket_name

## Create OpenSearch Serverless Collection

In [5]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
vector_store_name = 'idp-workshop'
index_name = "idp-workshop-rag"
encryption_policy_name = "idp-workshop-rag"
network_policy_name = "idp-workshop-rag"
access_policy_name = 'idp-workshop-rag'
identity = boto3.client('sts').get_caller_identity()['Arn']

aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')


In [ ]:
pp.pprint(collection)

In [ ]:
%store encryption_policy network_policy access_policy collection


In [ ]:
# Get the OpenSearch serverless collection URL
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)
# wait for collection creation
# This can take couple of minutes to finish
response = aoss_client.batch_get_collection(names=[vector_store_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    interactive_sleep(30)
    response = aoss_client.batch_get_collection(names=[vector_store_name])
print('\nCollection successfully created:')
pp.pprint(response["collectionDetails"])
# create opensearch serverless access policy and attach it to Bedrock execution role
try:
    create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role)
    # It can take up to a minute for data access rules to be enforced
    interactive_sleep(60)
    print("DONE")
except Exception as e:
    print("Policy already exists")
    pp.pprint(e)

## Create a vector index

In [6]:
# Create the vector index in Opensearch serverless, with the knn_vector field index mapping, specifying the dimension size, name and engine.
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1024,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [ ]:
# Create index
try:
    response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
    print('\nCreating index:')
    pp.pprint(response)

    # index creation can take up to a minute
    interactive_sleep(60)
except RequestError as e:
    # you can delete the index if its already exists
    # oss_client.indices.delete(index=index_name)
    print(f'Error while trying to create the index, with error {e.error}\nyou may unmark the delete above to delete, and recreate the index')
    

## Create Knowledge base

Steps:

* initialize Open search serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
  
* initialize chunking strategy, based on which KB will split the documents into pieces of size based on the headings and sections  using chunkingStrategyConfiguration.
  
* initialize the s3 configuration, which will be used to create the data source object later.
  
* initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.

### Document processing and chunking 

By leveraging bedrock knowledge base's [advanced parsing and hierarchial chunking methods](https://docs.aws.amazon.com/bedrock/latest/userguide/kb-chunking-parsing.html) we are able to automate the process of extracting complex information from tables, pictures and then chunk them based on the structure of the document 

#### Advanced Parsing

Advanced parsing involves breaking down unstructured documents into their constituent parts, such as text, tables, images, and metadata. 

You can use advanced parsing techniques for parsing non-textual information from supported file types. This feature allows you to select a FM for parsing complex data, such as tables, charts and images. 


#### Hierarchical Chunking

Hierarchical Chunking organizes chunks into a nested structure, where larger parent chunks contain smaller child chunks. This method has several advantages:

* Context Preservation: By maintaining a hierarchy, the system can preserve contextual relationships between chunks. This is particularly important when dealing with complex information that may require multiple levels of detail.

* Efficient Retrieval: When queries are made, the system can first retrieve child chunks for detailed information and then replace them with parent chunks for broader context. This approach enhances both precision and relevance in responses



By combining both of these techniques in the RAG, we are able to process complex documents that have tables and images but still retain the relationships between different sections of the document while retrieving it




In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# Ingest strategy - How to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "HIERARCHICAL",
    "hierarchicalChunkingConfiguration": {
        "levelConfigurations": [
            {
                "maxTokens": 1500
            },
            {
                "maxTokens": 300
            }
        ],
        "overlapTokens": 60
    }
}

parsingConfig = {
    "parsingConfiguration": {
                "bedrockFoundationModelConfiguration": {
                    "modelArn": f"arn:aws:bedrock:{region_name}::foundation-model/anthropic.claude-3-haiku-20240307-v1:0"
                },
                "parsingStrategy": "BEDROCK_FOUNDATION_MODEL"
            }
}

# The data source to ingest documents from, into the OpenSearch serverless knowledge base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v2:0"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Amazon data knowledge base."
roleArn = bedrock_kb_execution_role_arn

In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [ ]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [ ]:
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [ ]:
# Print the knowledge base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)
# keep the kb_id for invocation later 

%store kb_id

In [ ]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
pp.pprint(ds)

In [ ]:
%store ds kb

In [ ]:
#Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

### Document Extraction
We employ the Amazon 2024 10K report as an example document. We will first download the data and upload it into a S3 Bucket for us to process later. 

In [50]:
# Download and prepare dataset
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf',
]

filenames = [
    'Form-10k-2024.pdf',
]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

In [ ]:
# Upload data to s3 to the bucket that was configured as a data source to the knowledge base
s3_client = boto3.client("s3")
def uploadDirectory(path,bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucket_name,file)

uploadDirectory(data_root, bucket_name)

In [ ]:
%store bucket_name

In [ ]:
# Start an ingestion job
interactive_sleep(30)
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [ ]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

In [ ]:
# Get job 
while(job['status']!='COMPLETE' ):
    get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
    job = get_job_response["ingestionJob"]
    
    interactive_sleep(30)

pp.pprint(job)

In [ ]:
# Print the knowledge base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

### Searching using the knowledge base


Now we weill search for the data and utilize Claude 3 hakiku to do the final generation of the result

In [ ]:
# try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
# Lets see how different Anthropic Claude 3 models responds to the input text we provide
claude_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str) -> str:
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

In [ ]:
query = "Amazon net sales by geographical location and products in 2023?"

model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{claude_model_id}'
response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
generated_text = response['output']['text']
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {claude_model_id}:")
pp.pprint(generated_text )
print(f'---------- The citations for the response generated by {claude_model_id}:')
pp.pprint(contexts)
print()

## Retrieve API


Now using the retrieve API, we can see the raw responses of the chunks that we think are relevant to the question

In [ ]:
# retrieve api for fetching only the relevant context.
relevant_documents = bedrock_agent_runtime_client.retrieve(
    retrievalQuery= {
        'text': query
    },
    knowledgeBaseId=kb_id,
    retrievalConfiguration= {
        'vectorSearchConfiguration': {
            'numberOfResults': 3 # will fetch top 3 documents which matches closely with the query.
        }
    }
)

In [ ]:
pp.pprint(relevant_documents["retrievalResults"])